In [ ]:
# Download NLDAS Data

In [ ]:


import sys
import numpy as np
import pandas as pd
import rasterio
import refet
import pyproj as proj
import xarray
import requests
import multiprocessing as mp
import matplotlib.pyplot as plt
import configparser
import pathlib

In [ ]:
#sys.path.append("//")
sys.path.append("../../../Micromet")
import micromet
#from micromet.volk import ffp_climatology as ffp
#from micromet import AmerifluxDataProcessor

In [ ]:

# load initial flux data
#station = 'US-UTW'
#config_path = f'../../station_config/{station}.ini'
#config = configparser.ConfigParser()
#config.read(config_path)

spath = "../../secrets/config.ini"
sconfig = configparser.ConfigParser()
sconfig.read(spath)

ed_pass = sconfig['NLDAS']['pw']
ed_user = sconfig['NLDAS']['user']

In [ ]:
import datetime

#nldas_out_dir = Path('C:/Users/paulinkenbrandt/Documents/GitHub/MicroMet/Notebooks/Footprints')


for date in pd.date_range(start='2021-01-01', end='2025-02-01',freq='h'):
    hour = date.hour
    micromet.download_nldas(date,
                           hour,
                           ed_user,
                           ed_pass,)

In [ ]:
micromet.clip_to_utah_merge(file_dir="./NLDAS_data/", years = [2021,2022,2023,2024])


In [ ]:
micromet.calc_hourly_ffp_xr(input_data_dir=".",
                            years = [2021,2022,2023,2024],
                            output_dir="./output/")

In [ ]:
micromet.extract_nldas_xr_to_df([2021,2022,2023,2024], input_path="./output")

In [ ]:
output_path = pathlib.Path("./output/")
eto_df = pd.read_parquet(output_path / 'nldas_all.parquet')
normed_df = micromet.normalize_eto_df(eto_df)
normed_df.to_parquet(output_path / 'nldas_all_normed.parquet')

In [ ]:
output_path = pathlib.Path("./output/")

gpds = {}

for raster in output_path.glob('*weighted.tif'):

    date = pd.to_datetime(raster.stem.split('_')[0])
    gpds[f"{date:%Y-%m-%d}"] = micromet.outline_valid_cells(raster)

In [ ]:
footprints = pd.concat(gpds)
footprints['stationid'] = 'US-UTW'
footprints = footprints.reset_index().rename(columns={'level_0':'date'}).drop(['level_1'],axis=1)
footprints

In [ ]:
import configparser
config = configparser.ConfigParser()

config.read('../../secrets/config.ini')

from sqlalchemy import create_engine
import urllib.parse
host = config['DEFAULT']['ip']
pw = config['DEFAULT']['pw']
user = config['DEFAULT']['login']

encoded_password = urllib.parse.quote_plus(pw)

def postconn_et(encoded_password, host='localhost',user=user,port='5432',db='groundwater', schema = 'groundwater'):
    connection_text = "postgresql+psycopg2://{:}:{:}@{:}:{:}/{:}?gssencmode=disable".format(user,encoded_password,host,port,db)
    return create_engine(connection_text, connect_args={'options': '-csearch_path={}'.format(schema)})


engine = postconn_et(encoded_password, host=host, user=user)



In [ ]:
footprints.to_file("daily_footprint_areas.geojson",driver="GeoJSON")

In [ ]:
# 3. Export to PostGIS table
table_name = "daily_footprint_areas"
footprints.to_postgis(
    name=table_name,
    con=engine,
    if_exists="replace",   # or "append"
    index=False,           # do not write pandas index as a column
    #geom_col='geometry',     # use a custom geometry column name if desired
)

print(f"GeoDataFrame successfully exported to '{table_name}' in database.")